In [1]:
import urllib.request
import sqltables
import json
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModel

/Users/bob/venv/python3.11/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
db = sqltables.sqlite3.Database("submissions.sqlite3")
submissions = db.open_table("submissions")
    
if "specter" in list(db.tables):
    specter = db.open_table("specter")
else:
    specter = db.create_table(name="specter", column_names=["arxiv_id", "paper_info"])

In [3]:
processed = set(row.arxiv_id for row in specter)
len(processed)

8604

In [4]:
batch_size = 32

In [5]:
rows = list(submissions.view("""
select _.* from _ left join specter using (arxiv_id)
where specter.arxiv_id is null
group by arxiv_id
"""))
len(rows)

1535

In [6]:
tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
model = AutoModel.from_pretrained('allenai/specter')

for i in range(0, len(rows), batch_size):
    print(f"{i}/{len(rows)}")
# for i in [0]:
    batch_rows = rows[i:(i+batch_size)]
    title_abs = [row.title + tokenizer.sep_token + row.abstract for row in batch_rows]
    inputs = tokenizer(title_abs, padding=True, truncation=True, return_tensors="pt", max_length=512)
    result = model(**inputs)
    # take the first token in the batch as the embedding
    embeddings = result.last_hidden_state[:, 0, :]
    specter_rows = []
    for row, embedding in zip(batch_rows, embeddings.tolist()):
        paper_info = {"embedding": {"model": "specter@local", "vector": embedding}}
        specter_rows.append([row.arxiv_id, json.dumps(paper_info)])
    specter.insert(specter_rows)

/Users/bob/venv/python3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/bob/venv/python3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


0/1535
32/1535
64/1535
96/1535
128/1535
160/1535
192/1535
224/1535
256/1535
288/1535
320/1535
352/1535
384/1535
416/1535
448/1535
480/1535
512/1535
544/1535
576/1535
608/1535
640/1535
672/1535
704/1535
736/1535
768/1535
800/1535
832/1535
864/1535
896/1535
928/1535
960/1535
992/1535
1024/1535
1056/1535
1088/1535
1120/1535
1152/1535
1184/1535
1216/1535
1248/1535
1280/1535
1312/1535
1344/1535
1376/1535
1408/1535
1440/1535
1472/1535
1504/1535


In [7]:
db.close()